<a href="https://colab.research.google.com/github/snaiws/NLP_project/blob/RoBERTa%2Fbaseline/Model_roberta_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# transformer, datset 설치
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 9.5 MB/s 
     |████████████████████████████████| 346 kB 72.6 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 6.6 MB 57.8 MB/s 
     |████████████████████████████████| 212 kB 56.6 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 140 kB 37.2 MB/s 
     |████████████████████████████████| 1.1 MB 57.9 MB/s 
     |████████████████████████████████| 127 kB 58.4 MB/s 
     |████████████████████████████████| 144 kB 51.9 MB/s 
     |████████████████████████████████| 271 kB 62.4 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.

In [ ]:
# tensorboard 패키지 설치
!pip install jupyter-tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for jupyter-tensorboard: filename=jupyter_tensorboard-0.2.0-py2.py3-none-any.whl size=15258 sha256=67d836603526b5be9b94f8e75b8cf62161bdd882365b98078da34be2815ebb8d
  Stored in directory: /root/.cache/pip/wheels/59/14/ab/6d0bce449039ebdcbf45c1aff8c19153a68bf3f0492a32620e
Successfully built jupyter-tensorboard


In [ ]:
# tensorboard extension 설치
%load_ext tensorboard

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 9.7 MB/s 
     |████████████████████████████████| 81 kB 10.2 MB/s 
     |████████████████████████████████| 209 kB 68.6 MB/s 
     |████████████████████████████████| 78 kB 7.6 MB/s 
     |████████████████████████████████| 146 kB 55.3 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 112 kB 61.3 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=60e2bc1f7ce68d6e3c7055efe3b3a4d2508f61e794ab0a93c3eb2b939dbae7bc
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import transformers
from transformers import EarlyStoppingCallback
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader,  RandomSampler, SequentialSampler, random_split
from torch.nn.utils import clip_grad_norm_

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

from datasets import load_metric
from torch.utils.tensorboard import SummaryWriter
import os
import re
import pandas as pd
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt

In [ ]:
#reset gpu cache
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# seed
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [ ]:
# argument setting
train_batch_size = 58
eval_batch_size = 64
epochs=5
patience=2
lr=2.335281794951567e-05
model_checkpoint = "klue/roberta-base"

loss_fct = nn.CrossEntropyLoss()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

loading file https://huggingface.co/klue/roberta-base/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2
loading file https://huggingface.co/klue/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/klue/roberta-base/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23
loading file https://huggingface.co/klue/roberta-base/resolve/main/tokenizer_config.json from cache 

In [ ]:
cd /content/drive/MyDrive/AIBootCamp/NLP

/content/drive/MyDrive/AIBootCamp/NLP


In [ ]:
_CUR_DIR = os.path.abspath(os.curdir)
print(f"My current directory : {_CUR_DIR}")
_DATA_DIR = os.path.join(_CUR_DIR, "data/klue-sts-v1.1")
print(f"My data directory : {_DATA_DIR}")

My current directory : /content/drive/MyDrive/AIBootCamp/NLP
My data directory : /content/drive/MyDrive/AIBootCamp/NLP/data/klue-sts-v1.1


In [ ]:
from sklearn.model_selection import train_test_split

class Preprocess:
    def __init__(self,train,test):
        self.train=train
        self.test=test

    def check_st(self, text):
      text = re.sub('a-zA-Z一-龥㐀-䶵豈-龎[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘〈〉|\(\)\[\]\<\>`\'…》《]','', text)
      return text
    
    def robeta_base(self,dataset,mode=None):
      '''
      transform to the Input example
      '''
      #중복 제거
      dataset = dataset.drop_duplicates(['sentence1','sentence2']).reset_index(drop=True)

      sent_pairs = [sent1+' '+sent2 for sent1, sent2 in zip(dataset['sentence1'], dataset['sentence2'])]
      sent_pairs = list(map(self.check_st, sent_pairs))

      #real-label -> regression 활용 가능?
      r_score=[i['real-label'] for i in dataset['labels']]

      #custom label : int
      c_score=[round(i['real-label']) for i in dataset['labels']]

      #b_score label : [0,1] -> f1 scroe
      b_score = [i['binary-label'] for i in dataset['labels']]
      input_examples = pd.DataFrame({'guid': dataset['guid'], 'sent_pair' : sent_pairs, 'r_label' : r_score, 'c_label' : c_score, 'b_label' : b_score})

      if mode == 'train':
        train, valid = train_test_split(input_examples, test_size=0.1, random_state=seed)
        return train, valid

      return input_examples

In [ ]:
# Custom Dataset
class CustomDataset(Dataset):
    """
    - input_data: list of string
    - target_data: list of int
    """
    
    def __init__(self, input_data:list, target_data:list) -> None:
        self.X = input_data
        self.Y = target_data
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]

In [ ]:
# dataset = load_dataset('klue', task)
klue_sts_train = load_dataset("klue", "sts", split='train[:90%]')
klue_sts_valid = load_dataset("klue", "sts", split='train[-10%:]') # train의 10%를 validation set으로 사용
klue_sts_test = load_dataset("klue", "sts", split='validation')

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)
Reusing dataset klue (/root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


In [ ]:
# real-label을 label로 변경 
klue_sts_train = klue_sts_train.flatten()
klue_sts_train = klue_sts_train.rename_column('labels.real-label','label')
klue_sts_valid = klue_sts_valid.flatten()
klue_sts_valid = klue_sts_valid.rename_column('labels.real-label','label')
klue_sts_test = klue_sts_test.flatten()
klue_sts_test = klue_sts_test.rename_column('labels.real-label','label')

In [ ]:
# 최대 길이 설정 
max_length = 512

# 모델 입력으로 만들어 주는 전처리 함수 
# truncation으로 길이 넘어가면 자를 수 있게 설정, padding 으로 길이 맞춰 주기 
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'],
                     truncation=True, max_length=max_length, padding=True)


In [ ]:
# 전처리 함수 각 데이터셋에 적용 
encoded_train = klue_sts_train.map(preprocess_function, batched=True)
encoded_valid = klue_sts_valid.map(preprocess_function, batched=True)
encoded_test = klue_sts_test.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           num_labels=num_labels)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'class

In [ ]:
# 텐서보드 dir 정의 
logdir_path = os.path.join(_CUR_DIR, "sts_log")# log 파일을 저장할 경로를 지정
writer = SummaryWriter(logdir_path) 

In [ ]:
##### 베이스 라인 모델: batch_size= 32, 나머지는 default

output_dir = os.path.join(_CUR_DIR, "sts_output2")

In [ ]:
args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,

    # Model Save & Load
    save_strategy = "epoch", # 각 epoch 마지막에 저장 
    load_best_model_at_end=True, # train 종료시 best model 로드할지 여부

    # lerning_rate
    # learning_rate = lr,
    
    # Dataset
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    
    # Evaluation 
    evaluation_strategy = "epoch",# 각 epoch 마지막에 평가

    # Randomness
    seed=seed,
)

In [ ]:
# f1과 pearson 메트릭 불러오기 
metric_f1 = load_metric("f1") 
metric_pearson = load_metric("pearsonr") 

In [ ]:
# 출력시 메트릭 정의 하는 함수 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    predict_f1 = [1 if i>=3.0 else 0 for i in predictions] # 3.0 이상-> 1, 3.0 미만 -> 0으로 변환
    labels_f1 = [1 if i>=3.0 else 0 for i in labels] # 3.0 이상-> 1, 3.0 미만 -> 0으로 변환

    pr = metric_pearson.compute(predictions=predictions, # peason r 계산
                                   references=labels)

    f1 = metric_f1.compute(predictions=predict_f1,   # f1
                                     references=labels_f1)
    return {'pearsonr' : pr,
             'f1' : f1}

In [ ]:
#옵티마이져 및 스케줄러 설정:32
optimizer = AdamW(
    model.parameters(),
    lr=5e-5,
    eps=1e-8
)

batch_size = 32
total_steps = round(len(encoded_train.shard(index=1, num_shards=10))/batch_size)*5

# lr 선형으로 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# trainer 정의 
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train,
    eval_dataset=encoded_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers = (optimizer, scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience = patience)] # earlystopping 추가
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10501
  Num Epochs = 25
  Instantaneous batch size per device = 58
  Total train batch size (w. parallel, distributed & accumulation) = 58
  Gradient Accumulation steps = 1
  Total optimization steps = 4550


Epoch,Training Loss,Validation Loss,Pearsonr,F1
1,No log,0.201800,{'pearsonr': 0.9746447531475442},{'f1': 0.9565987599645704}
2,No log,0.201800,{'pearsonr': 0.9746447531475442},{'f1': 0.9565987599645704}
3,0.238000,0.201800,{'pearsonr': 0.9746447531475442},{'f1': 0.9565987599645704}


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64
Trainer is attempting to log a value of "{'pearsonr': 0.9746447531475442}" of type <class 'dict'> for key "eval/pearsonr" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9565987599645704}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /content/drive/MyDrive/AIBootCamp/NLP/sts_output2/checkpoi

TrainOutput(global_step=546, training_loss=0.230909757125072, metrics={'train_runtime': 392.5236, 'train_samples_per_second': 668.813, 'train_steps_per_second': 11.592, 'total_flos': 2039607796506498.0, 'train_loss': 0.230909757125072, 'epoch': 3.0})

In [ ]:
# baseline score
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 64


Trainer is attempting to log a value of "{'pearsonr': 0.9746447531475442}" of type <class 'dict'> for key "eval/pearsonr" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9565987599645704}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'epoch': 3.0,
 'eval_f1': {'f1': 0.9565987599645704},
 'eval_loss': 0.20179980993270874,
 'eval_pearsonr': {'pearsonr': 0.9746447531475442},
 'eval_runtime': 3.7256,
 'eval_samples_per_second': 313.236,
 'eval_steps_per_second': 5.1}

In [ ]:
# test set으로 prediction 
predictions = trainer.predict(encoded_test)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 519
  Batch size = 64


In [ ]:
# test set score 
predictions.metrics

{'test_f1': {'f1': 0.8346456692913387},
 'test_loss': 0.5009527802467346,
 'test_pearsonr': {'pearsonr': 0.8997988484485431},
 'test_runtime': 1.738,
 'test_samples_per_second': 298.616,
 'test_steps_per_second': 5.178}

# Hyper Parameter Tuning

In [ ]:
#hyperparamer search
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# trainer 여러번 돌기 때문에 모델 초기화 해줘야함 
def model_init():
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels) 
    
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels) 

In [ ]:
output_hyper_dir = os.path.join(_CUR_DIR, "sts_hyper")

hyper_args = TrainingArguments(
    # checkpoint
    output_dir=output_hyper_dir,

    # evaluation
    evaluation_strategy="epoch",

    #log
    logging_strategy="steps",
    logging_steps=1,
    logging_first_step=False,

    overwrite_output_dir=True,
    
    save_strategy="epoch",
    load_best_model_at_end=True,
    
    metric_for_best_model="eval_f1",

    seed=seed,

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# 하이퍼파라미터 튜닝때 사용할 매트릭스 함수 
def hyper_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    predict_f1 = [1 if i>=3.0 else 0 for i in predictions]
    labels_f1 = [1 if i>=3.0 else 0 for i in labels]

    f1 = metric_f1.compute(predictions=predict_f1,
                                     references=labels_f1)
    return f1

In [ ]:
num_labels = 1

In [ ]:
# 탐색할 파라미터 값 지정 
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 3e-5, log=True),
        "warmup_steps":  trial.suggest_float("warmup_steps", 0.01, 0.09, step=0.3),
        "weight_decay":  trial.suggest_float("weight_decay", 1e-6, 1e-1),   
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32]),
    }

In [ ]:
# trainer 정의 
trainer = Trainer(
    args=hyper_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_valid,
    model_init=model_init,
    tokenizer=tokenizer,
    compute_metrics=hyper_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience = patience)] # earlystopping 추가
)

loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "

In [ ]:
def my_objective(metrics):
    return metrics["eval_f1"]   

In [ ]:
# 탐색 시작 
best_run = trainer.hyperparameter_search(n_trials=2, 
                                         direction="maximize", 
                                         hp_space=my_hp_space,
                                         compute_objective=my_objective)

[I 2022-06-01 08:18:20,592] A new study created in memory with name: no-name-36d1304b-457b-465d-8863-448930d10fef
/usr/local/lib/python3.7/dist-packages/optuna/distributions.py:548: UserWarning: The distribution is specified by [0.01, 0.09] and q=0.3, but the range is not divisible by `q`. It will be replaced by [0.01, 0.01].
  low=low, old_high=old_high, high=high, step=q
Trial:
loading configuration file https://huggingface.co/klue/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,


Epoch,Training Loss,Validation Loss,F1
1,0.040700,0.180550,0.955816
2,0.045100,0.223150,0.952381
3,0.043300,0.176157,0.963295


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-0/checkpoint-1313
Configuration saved in /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-0/checkpoint-1313/config.json
Model weights saved in /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-0/checkpoint-1313/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-0/checkpoint-1313/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-0/checkpoint-1313/s

Epoch,Training Loss,Validation Loss,F1
1,0.036600,0.175341,0.959786
2,0.057100,0.221591,0.951456
3,0.047600,0.177857,0.961504


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-1/checkpoint-1313
Configuration saved in /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-1/checkpoint-1313/config.json
Model weights saved in /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-1/checkpoint-1313/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-1/checkpoint-1313/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/AIBootCamp/NLP/sts_hyper/run-1/checkpoint-1313/s

In [ ]:
# best model 파라미터 값 
best_run.hyperparameters

{'learning_rate': 1.539880137960369e-05,
 'per_device_train_batch_size': 16,
 'warmup_steps': 0.01,
 'weight_decay': 0.07028983728388466}

In [ ]:
tune_args = TrainingArguments(
    # checkpoint
    output_dir=output_dir,

    # Model Save & Load
    save_strategy = "epoch", # 'steps'
    load_best_model_at_end=True,

    # Dataset
    num_train_epochs=epochs,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    
    # Optimizer (AdamW)
    learning_rate= 1.539880137960369e-05, 
    warmup_steps=  0.01,
    weight_decay= 0.07028983728388466,

    # Evaluation 
    evaluation_strategy = "epoch",


    # Randomness
    seed= seed,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model,
    tune_args,
    train_dataset=encoded_train,
    eval_dataset=encoded_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

In [ ]:
# 학습 시작
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10501
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3285


Epoch,Training Loss,Validation Loss,Pearsonr,F1
1,0.167700,0.329344,{'pearsonr': 0.9739637572281786},{'f1': 0.9466316710411198}
2,0.141300,0.201549,{'pearsonr': 0.9783806667507823},{'f1': 0.9633601429848078}
3,0.106300,0.147764,{'pearsonr': 0.9798616752331278},{'f1': 0.9648964896489649}
4,0.059800,0.191242,{'pearsonr': 0.9803258649831558},{'f1': 0.9609929078014184}


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 32
Trainer is attempting to log a value of "{'pearsonr': 0.9739637572281786}" of type <class 'dict'> for key "eval/pearsonr" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9466316710411198}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to /content/drive/MyDrive/AIBootCamp/NLP/sts_output2/checkpoi

TrainOutput(global_step=2628, training_loss=0.10827282883079448, metrics={'train_runtime': 603.7904, 'train_samples_per_second': 86.959, 'train_steps_per_second': 5.441, 'total_flos': 2707594004554080.0, 'train_loss': 0.10827282883079448, 'epoch': 4.0})

In [ ]:
# score
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1167
  Batch size = 32


Trainer is attempting to log a value of "{'pearsonr': 0.9798616752331278}" of type <class 'dict'> for key "eval/pearsonr" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9648964896489649}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'epoch': 4.0,
 'eval_f1': {'f1': 0.9648964896489649},
 'eval_loss': 0.14776434004306793,
 'eval_pearsonr': {'pearsonr': 0.9798616752331278},
 'eval_runtime': 4.9662,
 'eval_samples_per_second': 234.988,
 'eval_steps_per_second': 7.45}

In [ ]:
# test set으로 prediction 
predictions = trainer.predict(encoded_test)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: labels.binary-label, guid, sentence2, sentence1, labels.label, source. If labels.binary-label, guid, sentence2, sentence1, labels.label, source are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 519
  Batch size = 32


In [ ]:
predictions.metrics

{'test_f1': {'f1': 0.8512396694214875},
 'test_loss': 0.33514708280563354,
 'test_pearsonr': {'pearsonr': 0.9250721379209141},
 'test_runtime': 1.8916,
 'test_samples_per_second': 274.37,
 'test_steps_per_second': 8.987}

In [ ]:
{'test_f1': {'f1': 0.8346456692913387},
 'test_loss': 0.5009527802467346,
 'test_pearsonr': {'pearsonr': 0.8997988484485431},
 'test_runtime': 1.738,
 'test_samples_per_second': 298.616,
 'test_steps_per_second': 5.178}

In [ ]:
# result
result= pd.DataFrame({'test_f1':[round(0.8512396694214875, 4),round(0.8346456692913387, 4),round(0.814498933901919, 4)],
                             'test_loss':[ round(0.33514708280563354,4), round(0.5009527802467346, 4), round(0.4114389419555664, 4)],
                             'test_pearsonr':[round(0.9250721379209141, 4), round(0.8997988484485431, 4), round(0.9010864050775359, 4)],
                             'test_runtime':[1.8916,1.738,3.0923]}).T
result.columns = ['hyper', 'optimizer', 'base']
result

,hyper,optimizer,base
test_f1,0.8512,0.8346,0.8145
test_loss,0.3351,0.5010,0.4114
test_pearsonr,0.9251,0.8998,0.9011
test_runtime,1.8916,1.7380,3.0923


# save score 

In [75]:
predictions[0]

array([[ 4.653361  ],
       [ 1.7810117 ],
       [ 1.4303722 ],
       [ 4.100582  ],
       [ 2.887404  ],
       [ 1.7765844 ],
       [ 3.6381445 ],
       [ 0.89057696],
       [ 0.04111077],
       [ 1.714973  ],
       [ 3.7967744 ],
       [ 2.7179408 ],
       [ 0.53205067],
       [ 2.452306  ],
       [ 4.462683  ],
       [ 3.402286  ],
       [ 4.5356064 ],
       [ 1.4683228 ],
       [ 0.06471527],
       [ 0.79742837],
       [ 3.5034406 ],
       [ 3.6744692 ],
       [ 4.072261  ],
       [ 3.863794  ],
       [ 0.01405839],
       [ 3.6003497 ],
       [ 3.279758  ],
       [ 2.6536767 ],
       [ 0.40689546],
       [ 3.6236727 ],
       [ 1.6346698 ],
       [ 0.07456977],
       [ 0.27325472],
       [ 3.6134775 ],
       [ 4.639293  ],
       [ 2.2422178 ],
       [ 3.5118694 ],
       [ 3.8391118 ],
       [ 4.7428346 ],
       [ 3.2471724 ],
       [ 3.4478896 ],
       [ 4.4284425 ],
       [ 1.096538  ],
       [ 2.968014  ],
       [ 2.110668  ],
       [ 3

In [ ]:
predict = [1 if i>=3.0 else 0 for i in predictions[0]]
predict

In [64]:
encoded_test

Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels.label', 'label', 'labels.binary-label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 519
})

In [79]:
score = pd.DataFrame({'guid' : encoded_test['guid'],
                      'true_real_label' : encoded_test['label'],
                      'true_binary_label' : encoded_test['labels.binary-label'],
})
score

,guid,true_real_label,true_binary_label
0,klue-sts-v1_dev_00000,4.857143,1
1,klue-sts-v1_dev_00001,1.428571,0
2,klue-sts-v1_dev_00002,1.285714,0
3,klue-sts-v1_dev_00003,3.714286,1
4,klue-sts-v1_dev_00004,2.500000,0
...,...,...,...
514,klue-sts-v1_dev_00514,2.200000,0
515,klue-sts-v1_dev_00515,2.833333,0
516,klue-sts-v1_dev_00516,0.333333,0
517,klue-sts-v1_dev_00517,0.333333,0


In [80]:
score['predict_real_label'] = predictions[0]

In [82]:
score['predict_binary_label'] = score.predict_real_label.apply(lambda x: 1 if x>=3.0 else 0)

In [84]:
score

,guid,true_real_label,true_binary_label,predict_real_label,predict_binary_label
0,klue-sts-v1_dev_00000,4.857143,1,4.653361,1
1,klue-sts-v1_dev_00001,1.428571,0,1.781012,0
2,klue-sts-v1_dev_00002,1.285714,0,1.430372,0
3,klue-sts-v1_dev_00003,3.714286,1,4.100582,1
4,klue-sts-v1_dev_00004,2.500000,0,2.887404,0
...,...,...,...,...,...
514,klue-sts-v1_dev_00514,2.200000,0,3.613109,1
515,klue-sts-v1_dev_00515,2.833333,0,3.170947,1
516,klue-sts-v1_dev_00516,0.333333,0,1.800307,0
517,klue-sts-v1_dev_00517,0.333333,0,0.476707,0


In [85]:
score.to_csv('RoBERTa_hyper_dev_set_score.csv', index=False)